In [3]:
import psycopg2
import glob

In [4]:
# Try connection to the database 'carto'
try:
    #conn for connection
    conn =  psycopg2.connect(dbname='carto', user='carto', password='carto', host='postgis', port='5432')
except psycopg2.DatabaseError:
    print ("I am unable to connect the database")

In [5]:
# A query to create table for NY taxi data
q_create_table = """
                 DROP TABLE IF EXISTS taxi;
                 CREATE TABLE taxi
                 (
                         VendorID bytea,
                         tpep_pickup_datetime timestamp,
                         tpep_dropoff_datetime timestamp,
                         passenger_count int,
                         trip_distance numeric,
                         pickup_point geography(Point),
                         RateCodeID int,
                         store_and_fwd_flag bytea,
                         dropoff_point geography(Point),
                         payment_type int,
                         fare_amount numeric,
                         extra bytea,
                         mta_tax numeric,
                         tip_amount numeric,
                         tolls_amount numeric,
                         improvement_surcharge numeric,
                         total_amount numeric
                )
                """


In [6]:
try:
    cur = conn.cursor()
    cur.execute(q_create_table)
    conn.commit()
    print('Table created')

except psycopg2.DatabaseError:
    print ("Failed to create the table")

Table created


In [6]:
# Create the list of NY taxi data files


In [58]:
q_copy_data = '''
              copy taxi
              from'../data/NY_taxi_data/yellow_tripdata_2015-01_00.txt' delimiter ',' csv header;
              '''

try:
    cur.execute(q_copy_data)
    conn.commit()
    print('Data copied to the table')

except psycopg2.DatabaseError:
    print ("Failed to copy the data")

Failed to copy the data


In [7]:
try:
    with open('../data/NY_taxi_data/yellow_tripdata_2015-01_01', 'r') as file:
        rows = [tuple(line.strip().split(',')) for line in file]
        
        query = f'''insert into taxi 
                  (VendorID,
                   tpep_pickup_datetime,
                   tpep_dropoff_datetime,
                   passenger_count,
                   trip_distance,
                   pickup_point,
                   RateCodeID,
                   store_and_fwd_flag,
                   dropoff_point,
                   payment_type,
                   fare_amount,
                   extra,
                   mta_tax,
                   tip_amount,
                   tolls_amount,
                   improvement_surcharge,
                   total_amount)
                  values (%s,%s,%s,%s,%s, %s,%s,%s,%s,%s, %s,%s,%s,%s,%s, %s,%s,%s,%s)'''
        
        for row in rows:
            row[5] = f'point({row[5]} {row[6]})'
            row[10] = f'point({row[10]} {row[11]})'
            row.pop(6)
        #     # row.pop()
        #     # point2 = f'point({row[6]} {row[7]})'
            print(row)
        #     # cur.execute(query, row)
        #     break
        # print(query)
    # conn.commit()
    print('Data inserted to the table')

except psycopg2.DatabaseError:
    print ("Failed to copy data to the table")

TypeError: 'tuple' object does not support item assignment

In [ ]:
#close connection for this client
if conn:
    conn.close ()